In [16]:
import os
import shutil
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


In [5]:
import sys
sys.path.append('/Users/nathan/Code/CardCombo/')

In [6]:
from cardcombo.simulate import *
from cardcombo.plot import plot_num_of_cards, radar_plot

In [51]:
import os, pickle, re
import numpy as np
import pandas as pd
from multiprocessing import Pool

COLUMNS = ['flights', 'hotel', 'grocery', 'gas', 'dining', 'other']

def calculate_rewards(db, expenses):
    """
    A function to calculate the max rewards for a parsed dataframe.
    TODO: this needs to eventually include net_fee in it's equation.
    
    Inputs:
    - rewards_df (pd.DateFrame), a sliced dataframe with columns
        similar to to_plot
    - expenses (dict), a dictionary containing all of a person's 
        annual expenses
        
    Returns:
    - cash_rewards (float), the annual rewards one would recieve
    """
    cash_rewards = 0
    for category, expense in expenses.items():
        try:
            if isinstance(db[category], float):
                rate = db[category] / 100
            else:
                rate = max(db[category]) / 100
            cash_rewards += rate * expense
        except:
            continue
        
    return cash_rewards

def simulate(db, expenses, num_of_cards):

    db_copy = db.copy()
    db_copy['sum_of'] = db[COLUMNS].sum(axis=1)

    # Only looking at credit card with greater than 1% back at everything
    db_copy = db_copy[db_copy.sum_of > len(COLUMNS)]

    # Removing cards that have a weirdly high point value
    # These data are either parsed incorrectly or are hotel rewards were
    # the point to cent ratio is high (e.g. many points to a cent)
    db_copy = db_copy[db_copy.sum_of <= 2*len(COLUMNS)] 

    card_rewards = []
    cloud_list = []

    for i in range(1000):
        choice = sorted(np.random.choice(db_copy.index, num_of_cards))
        card_rewards.append(calculate_rewards(db_copy.loc[choice], expenses))
        cloud_list.append((calculate_rewards(db_copy.loc[choice], expenses), choice))
    return card_rewards, cloud_list

def simulate_all(db, expenses, range_of_cards=7):
    calculated_rewards = {}
    word_cloud = {}
    for num_of_cards in range(range_of_cards+1):
        card_rewards, cloud_list = simulate(db, expenses, num_of_cards)
        calculated_rewards[num_of_cards] = card_rewards
        word_cloud[num_of_cards] = cloud_list
    return calculated_rewards, word_cloud



example_expenses = {'flights': 895.4899999999999,
    'hotel': 9.9,
    'grocery': 1803.92,
    'gas': 336.03999999999996,
    'utilities': 0.0,
    'restaurants': 4069.8500000000004,
    'other': 0.240000000005}


In [7]:
db = pickle.load(open('../db.pkl', 'rb'))
db.head()

,flights,hotel,grocery,gas,dining,other,rotating,flat_cash_back,req_credit,annual_fee,card_type,application_link,review_link,annual_bonus,image_url,converted_name
name,,,,,,,,,,,,,,,,
Alliant Cashback Visa® Signature Credit Card,0.0,0.0,0.0,0.0,0.0,0.0,False,False,690,99.0,Visa,NaN,https://www.nerdwallet.com/reviews/credit-card...,0.0,//cdn.nerdwallet.com/img/cc/CC_placeholder.svg,Alliant-Cashback-Visa-Signature-Credit-Card
American Express Cash Magnet® Card,1.5,1.5,1.5,1.5,1.5,1.5,False,True,690,0.0,American Express,NaN,https://www.nerdwallet.com/reviews/credit-card...,0.0,//cdn.nerdwallet.com/img/cc/CC_placeholder.svg,American-Express-Cash-Magnet-Card
Capital One® Quicksilver® Cash Rewards Credit Card,1.5,1.5,1.5,1.5,1.5,1.5,False,True,690,0.0,Unknown,NaN,https://www.nerdwallet.com/reviews/credit-card...,0.0,//cdn.nerdwallet.com/img/cc/CC_placeholder.svg,Capital-One-Quicksilver-Cash-Rewards-Credit-Card
Chase Freedom Unlimited®,1.5,1.5,1.5,1.5,1.5,1.5,False,True,690,0.0,Unknown,https://www.nerdwallet.com/redirect/credit-car...,https://www.nerdwallet.com/reviews/credit-card...,0.0,//www.nerdwallet.com/c1/images/3214_3ad108f06f...,Chase-Freedom-Unlimited
Citi® Double Cash Card – 18 month BT offer,2.0,2.0,2.0,2.0,2.0,2.0,False,True,720,0.0,Unknown,https://www.nerdwallet.com/redirect/credit-car...,https://www.nerdwallet.com/reviews/credit-card...,0.0,//www.nerdwallet.com/c1/images/4220_3fb5399449...,Citi-Double-Cash-Card-–-18-month-BT-offer


In [8]:
example_expenses = {'flights': 895.4899999999999,
    'hotel': 9.9,
    'grocery': 1803.92,
    'gas': 336.03999999999996,
    'utilities': 0.0,
    'restaurants': 4069.8500000000004,
    'other': 0.240000000005}

In [9]:
_, results = simulate_all(db, example_expenses)


{0: [(0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, []),
  (0.0, [

In [10]:
df = pd.DataFrame(_)

In [113]:
import random
import textwrap
df_results = []
for num_cards, num_results in results.items():
    for num_result in num_results:
        text = 'Number of Cards: {}<br>Net Rewards: $ {}<br>Cards:<br>- {}'.format(
            num_cards, round(num_result[0],2), "<br>- ".join(num_result[1])
        )
        df_results.append([num_cards, text] + list(num_result))
        
        
df = pd.DataFrame(df_results, columns=['num_cards', 'text','net_rewards', 'cards'])
df.cards = df.cards.map(lambda x: ',\n'.join(x)[:-3])
df['rand_x'] = df.num_cards.map(lambda x: x + random.gauss(0, 0.1))
#df["text"].apply(
#    lambda t: "<br>".join(textwrap.wrap(t))
#)

In [114]:
'Number of Cards: {}\nNet Rewards: $ {}\nCards:{}'.format(
            num_cards,
            round(num_result[0],2), 0
            "\n\t- ".join(num_results[1])
        )

SyntaxError: invalid syntax (<ipython-input-114-ba7fc6b47151>, line 4)

In [115]:
df

,num_cards,text,net_rewards,cards,rand_x
0,0,Number of Cards: 0<br>Net Rewards: $ 0.0<br>Ca...,0.0000,,-0.129356
1,0,Number of Cards: 0<br>Net Rewards: $ 0.0<br>Ca...,0.0000,,0.144950
2,0,Number of Cards: 0<br>Net Rewards: $ 0.0<br>Ca...,0.0000,,0.003885
3,0,Number of Cards: 0<br>Net Rewards: $ 0.0<br>Ca...,0.0000,,0.156562
4,0,Number of Cards: 0<br>Net Rewards: $ 0.0<br>Ca...,0.0000,,-0.016744
...,...,...,...,...,...
19995,9,Number of Cards: 9<br>Net Rewards: $ -318.67<b...,-318.6715,Capital One® SavorOne® Cash Rewards Credit Car...,9.137843
19996,9,Number of Cards: 9<br>Net Rewards: $ -465.13<b...,-465.1333,"British Airways Visa Signature® Card,\nCapital...",8.973393
19997,9,Number of Cards: 9<br>Net Rewards: $ -34.08<br...,-34.0810,"Amazon Business Prime American Express Card,\n...",8.885607
19998,9,Number of Cards: 9<br>Net Rewards: $ -910.12<b...,-910.1173,"American Express® Business Gold Card,\nDiscove...",9.001857


In [285]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random

def plot_num_of_cards(results):

    df_results = []
    for num_cards, num_results in results.items():
        for num_result in num_results:
            text = 'Number of Cards: {}<br>Net Rewards: $ {}<br>Cards:<br>- {}'.format(
                num_cards, round(num_result[0],2), "<br>- ".join(num_result[1])
            )
            df_results.append([num_cards, text] + list(num_result))


    df = pd.DataFrame(df_results, columns=['num_cards', 'text','net_rewards', 'cards'])
    df.cards = df.cards.map(lambda x: ',\n'.join(x)[:-3])
    df['rand_x'] = df.num_cards.map(lambda x: x + random.gauss(0, 0.1))

    fig = go.FigureWidget(
        make_subplots(rows=2, cols=1, specs=[[{"type": "scatter"}], [{'type': 'scatterpolar'}]]))

    fig.add_trace(
        go.Scatter(
            name='Aggrigate Net Rewards',
            x=df['rand_x'], 
            y=df['net_rewards'], 
            mode='markers', 
            text=df['text'],
            hovertemplate='%{text}',
            marker=dict(
                color='mintcream',
                size=5,
                opacity=0.4, 
                line=dict(
                    color='Black',
                    width=0.5
                ))
        ),
        row=1, col=1
    )

    scatter = fig.data[0]

    text = None

    def update_point(trace, points, selector):
        new_data = []
        for d in fig.data:
            if d._plotly_name == 'scatterpolar':
                continue
            new_data.append(d)

        fig.data = new_data

        c = np.full_like(trace.text, 'mintcream')
        s = np.full_like(trace.text, 5)
        o = np.full_like(trace.text, 0.4)

        i = points.point_inds[0]
        c[i] = 'gold'
        s[i] = 15
        o[i] = 1

        with fig.batch_update():
            scatter.marker.color = list(c)
            scatter.marker.size = list(s)
            scatter.marker.opacity = list(o)

        text = trace.text[i]
        cards = text.split('Cards:<br>- ')[-1].split('<br>- ')
        series = []
        for card in cards:
            series.append(db.loc[card])

        radar = radar_plot(series)
        for t in radar.data:
            fig.add_trace(t, row=2, col=1)

    fig.add_trace(
        go.Scatter(x=df.num_cards.unique(), y=df.groupby('num_cards').max()['net_rewards'],
                   mode='lines',
                   name='Maximum Rewards', line=dict(color='#46039f')), row=1, col=1)
    fig.add_trace(
        go.Scatter(x=df.num_cards.unique(), y=df.groupby('num_cards').median()['net_rewards'],
                   mode='markers',
                   name='Median Rewards', marker=dict(color='#bd3786', size=8, line=dict(color='black', width=0.5))),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.num_cards.unique(), y=df.groupby('num_cards').min()['net_rewards'],
                   mode='lines', name='Minimum Rewards', line=dict(color='#ed7953')), row=1, col=1)



    scatter.on_click(update_point)

    fig.update_layout(
        title="Custom Credit Card Profile",
        xaxis_title="Number of Credit Cards",
        yaxis_title="Annual Rewards Earned ($)",
        legend_title="Legend Title",
        width=1000, height=1000
    )
    return fig

In [286]:
plot_num_of_cards(results)

FigureWidget({
    'data': [{'hovertemplate': '%{text}',
              'marker': {'color': 'mintcream', 'line'…